In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
import re

Read listings.csv

In [ ]:
Data = pd.read_csv("/content/listings.csv")

In [ ]:
Data.head()

Read review.csv files. Since the original file was too big to read, I split the file into 6 files

In [ ]:
df = pd.read_csv("/content/review.csv")
reviews = pd.concat(df)
reviews.head()

In [ ]:
Data.info()

In [ ]:
Data.columns

In [ ]:
Data.drop("source", axis = 1, inplace = True)
Data.drop("name", axis = 1, inplace = True)
Data.drop("description", axis = 1, inplace = True)
Data.drop("neighborhood_overview", axis = 1, inplace = True)
Data.drop("host_id", axis = 1, inplace = True)
Data.drop("host_about", axis = 1, inplace = True)
Data.drop("host_name", axis = 1, inplace = True)
Data.drop("host_since", axis = 1, inplace = True)
Data.drop("host_location", axis = 1, inplace = True)

In [ ]:
Data.drop("host_neighbourhood", axis = 1, inplace = True)
Data.drop("host_listings_count", axis = 1, inplace = True)
Data.drop("host_total_listings_count", axis = 1, inplace = True)
Data.drop("neighbourhood", axis = 1, inplace = True)
Data.drop("neighbourhood_cleansed", axis = 1, inplace = True)
Data.drop("property_type", axis = 1, inplace = True)
Data.drop("room_type", axis = 1, inplace = True)
Data.drop("bathrooms_text", axis = 1, inplace = True)
Data.drop("host_acceptance_rate", axis = 1, inplace = True)
Data.drop("instant_bookable", axis = 1, inplace = True)
Data.drop("host_verifications", axis = 1, inplace = True)
Data.drop("amenities", axis = 1, inplace = True)

In [ ]:
reviews.info()

In [ ]:
reviews.columns

In [ ]:
reviews.drop("id", axis = 1, inplace = True)
reviews.drop("date", axis = 1, inplace = True)
reviews.drop("reviewer_id", axis = 1, inplace = True)
reviews.drop("reviewer_name", axis = 1, inplace = True)

In [ ]:
reviews.info()

In [ ]:
reviews['listing_id'] = reviews.listing_id.astype('int')

In [ ]:
reviews.info()

Remove nan

In [ ]:
reviews = reviews[reviews['comments'].notna]

Sentiment Analysis

In [ ]:
!pip install langdetect

In [ ]:
def sentiment_analysis(data, x, new_column):
    data[new_column] = 0
    sentiment = SentimentIntensityAnalyzer()

    for i in range(len(data)):
        s = str(data.iloc[i, data.columns.get_loc(x)])

        sentiment_value = sentiment.polarity_scores(s)
        data.iloc[i, data.columns.get_loc(new_column)] = sentiment_value["compound"]

In [ ]:
sentiment_analysis(reviews, 'comments', 'sentiment_score')

In [ ]:
reviews.head(100)

In [ ]:
reviews.drop("comments", axis = 1, inplace = True)

In [ ]:
new_reviews = reviews

In [ ]:
new_reviews = new_reviews.set_index('listing_id')

In [ ]:
new_reviews

In [ ]:
new_reviews = new_reviews.groupby(level = 'listing_id').mean()
new_reviews

In [ ]:
new_Data = Data.set_index('id')
new_Data

In [ ]:
Total = pd.merge(new_Data, new_reviews, left_index = True, right_index = True)

In [ ]:
Total

In [ ]:
new_reviews.drop('comments', axis = 1, inplace = True)

In [ ]:
Total.to_csv("total.csv")

In [ ]:
Total.columns

In [ ]:
Total = pd.read_csv('total.csv')

In [ ]:
Total.head()

Remove the comments written by non-english

In [ ]:
from langdetect import detect

In [ ]:
def remove_non_english(data, x):
    wrong_index = []
    for i in range(len(data)):
      try:
        if detect(data.iloc[i, data.columns.get_loc(x)]) != 'en':
            wrong_index.append(i)
      except:
        wrong_index.append(i)

    data_drop = data.drop(wrong_index)
    data_drop = data_drop.reset_index(drop = True)
    return data_drop

In [ ]:
new_reviews = reviews

In [ ]:
new_reviews.info()

In [ ]:
en_reviews = remove_non_english(new_reviews, 'comments')

In [ ]:
en_reviews.to_csv('en_reviews.csv')

Define the function to convert non-numeric data into numeric data

In [ ]:
def boolean(x):
  if x == 'f':
    return -1
  return 1

def host_response(x):
  if x == 'within an hour':
    return 10
  elif x == 'within a few hours':
    return 8
  elif x == 'within a day':
    return 6
  else:
    return 4

def fillnamode(x):
  Total.loc[:, x] = Total.loc[:, x].fillna(Total.loc[:, x].value_counts().index[0])
  return

def fillnamean(x):
  Total.loc[:, x] = Total.loc[:, x].fillna(Total.loc[:, x].mean())
  return

In [ ]:
for i in range(len(Total)):
    Total.iloc[i, Total.columns.get_loc('price')] = float(str(Total.iloc[i, Total.columns.get_loc('price')]).replace("$", "").replace(",", ""))
for i in range(len(Total)):
    Total.iloc[i, Total.columns.get_loc('host_response_rate')] = float(str(Total.iloc[i, Total.columns.get_loc('host_response_rate')]).strip("%"))/100

In [ ]:
fillnamean('review_scores_accuracy')
fillnamean('review_scores_cleanliness')
fillnamean('review_scores_checkin')
fillnamean('review_scores_communication')
fillnamean('review_scores_location')
fillnamean('review_scores_value')
fillnamean('bedrooms')
fillnamean('beds')

fillnamode('host_response_time')

In [ ]:
Total["host_is_superhost"] = Total['host_is_superhost'].map(boolean)
Total['host_has_profile_pic'] = Total['host_has_profile_pic'].map(boolean)
Total['host_identity_verified'] = Total['host_identity_verified'].map(boolean)
Total['host_response_time'] = Total['host_response_time'].map(host_response)

In [ ]:
super = Total['host_is_superhost']
profile = Total['host_has_profile_pic']
identity = Total['host_identity_verified']
rating = Total['review_scores_value']
response = Total['host_response_time']
amenity = Total['n_amenities']
num_review = Total['number_of_reviews']
sentiment = Total['sentiment_score']
price = Total['price']

In [ ]:
mean_price = price.mean()
std_price = price.std()
normal_price = (price - mean_price)/std_price
Total['price'] = (Total['price'] - mean_price)/std_price

In [ ]:
print((profile < 0).sum())
print((profile > 0).sum())
print((identity < 0).sum())
print((identity > 0).sum())

In [ ]:
plt.scatter(identity, rating, marker = '+', s = 10, c = 'red')
plt.xlabel("identity verification")
plt.ylabel("reveiw score value")

In [ ]:
plt.scatter(amenity, rating, marker = '+', s = 10, c = 'red')
plt.xlabel("number of amenities")
plt.ylabel("reveiw score value")

In [ ]:
plt.scatter(num_review, rating, marker = '+', s = 10 , c = 'red')
plt.xlabel("number of reviews")
plt.ylabel("reveiw score value")

In [ ]:
plt.scatter(sentiment, rating, marker = '+', s = 10, c = 'red')
plt.xlabel("sentiment score")
plt.ylabel("reveiw score value")

In [ ]:
plt.scatter(normal_price, rating,  marker = '+', s = 10, c = 'red')
plt.xlabel("normalised price")
plt.ylabel("reveiw score value")

In [ ]:
plt.scatter(response, rating, s = 10, c = 'red')
plt.xlabel("Host reponse time")
plt.ylabel("reveiw score value")

In [ ]:
Total.info()

In [ ]:
Total.drop('host_response_rate', axis = 1, inplace = True)

In [ ]:
New = Total

In [ ]:
pd.isnull(New).sum()

In [ ]:
Total.columns

In [ ]:
label1 = [1, 2, 3, 4, 5]
label2 = [1, 2, 3, 4]
label3 = [1, 2, 3, 4, 5, 6]

In [ ]:
def bin_scores(data, bin, score, labels):
    data[bin] = pd.qcut(data[score], q=6, duplicates='drop', labels = labels)

In [ ]:
Total.columns

In [ ]:
bin_scores(Total,'rating','review_scores_rating',label1)
bin_scores(Total,'accuracy','review_scores_accuracy',label1)
bin_scores(Total,'cleanliness','review_scores_cleanliness',label1)
bin_scores(Total,'checkin','review_scores_checkin',label2)
bin_scores(Total,'communication','review_scores_communication',label2)
bin_scores(Total,'location','review_scores_location',label1)
bin_scores(Total,'value','review_scores_value',label3)

In [ ]:
target = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
new_target = ['rating', 'accuracy', 'cleanliness', 'checkin', 'communication', 'location', 'value']

Total = Total.drop(columns = target)
Total.head()

In [ ]:
correlation = Total.corr()

In [ ]:
fig, ax = plt.subplots(figsize = (12, 10))
sns.heatmap(correlation, square = True, annot = True, xticklabels = correlation.columns.values, yticklabels = correlation.columns.values)
plt.xlabel('Name of features')
plt.ylabel("Name of features")

In [ ]:
X = Total.drop(columns = ['Unnamed: 0']).drop(columns = new_target)
y = Total[new_target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
def main_model(model, Xtrain, ytrain):
    y_pred = model.predict(Xtrain)
    y_proba = model.predict_proba(Xtrain)

    acc = metrics.accuracy_score(ytrain, y_pred)
    pre = metrics.precision_score(ytrain, y_pred, average = 'macro', zero_division=0)
    recall = metrics.recall_score(ytrain, y_pred, average = 'macro')
    f1 = metrics.f1_score(ytrain, y_pred, average = 'macro')
    roc_auc = metrics.roc_auc_score(ytrain, y_proba, average = 'macro', multi_class = 'ovr')

    return acc, pre, recall, f1, roc_auc

def model_result(name, model, Xtrain, ytrain, Xtest, ytest):
    result = pd.DataFrame(columns = ('Model', 'dataset', 'Accuracy', 'Precision', 'Recall', 'F1 score', 'ROC_AUC'))

    model.fit(Xtrain, ytrain)
    acc, p, r, f1, roc_auc_train = main_model(model, Xtrain, ytrain)
    result.loc[0] = [name, 'train', acc, p, r, f1, roc_auc_train]

    acc, p, r, f1, roc_auc_test = main_model(model, Xtest, ytest)
    result.loc[1] = [name, 'test', acc, p, r, f1, roc_auc_test]
    return result

In [ ]:
def dummy_model(model, Xtrain, ytrain):
    y_pred = model.predict(Xtrain)

    acc = metrics.accuracy_score(ytrain, y_pred)
    pre = metrics.precision_score(ytrain, y_pred, average ='macro', zero_division=0)
    recall = metrics.recall_score(ytrain, y_pred, average ='macro')
    f1 = metrics.f1_score(ytrain, y_pred, average='macro')

    return acc, pre, recall, f1

In [ ]:
def dummy_result(name, model, Xtrain, ytrain, Xtest, ytest):
    result = pd.DataFrame(columns =('Model', 'dataset', 'Accuracy', 'Precision', 'Recall', 'F1 score'))

    model.fit(Xtrain, ytrain)
    acc, p, r, f1 = dummy_model(model, Xtrain, ytrain)
    result.loc[0] = [name, 'train', acc, p, r, f1]

    acc, p, r, f1 = dummy_model(model, Xtest, ytest)
    result.loc[1] = [name, 'test', acc, p, r, f1]
    return result

Logistic Regression

In [ ]:
model1 = LogisticRegression()
param_grid1 = {'C':[0.001, 0.01, 0.1, 1, 10], 'max_iter':[1000], 'solver':['saga']}

In [ ]:
Ci = []

for i in range(7):
  target = new_target[i]

  gs = GridSearchCV(LogisticRegression(), param_grid = param_grid1, scoring = 'accuracy', n_jobs = -1)
  gs = gs.fit(X_train, y_train[target])
  param = gs.best_params_
  Ci.append(param)

In [ ]:
Ci

In [ ]:
for i in range(7):
  target = new_target[i]
  parameter = Ci[i]
  model = LogisticRegression(**parameter)

  result = model_result('Logistic', model, X_train, y_train[target], X_test, y_test[target])
  print(result)

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
gs = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 0),
                  param_grid = [{'max_depth' : [1, 2, 3, 4, 5, 6, 7, None]}],
                  scoring = 'accuracy', cv = 2)

In [ ]:
psvc = []

for i in range(7):
  target = new_target[i]

  gs = gs.fit(X_train, y_train[target])
  param = gs.best_params_
  psvc.append(param)

In [ ]:
psvc

In [ ]:
psvc1 = [{'kernel': 'linear', 'C': 1.0},
 {'kernel': 'linear', 'C': 1.0},
 {'kernel': 'linear', 'C': 10.0},
 {'kernel': 'linear', 'C': 0.001},
 {'kernel': 'linear', 'C': 0.001},
 {'kernel': 'linear', 'C': 1.0},
 {'kernel': 'linear', 'C': 10.0}]

In [ ]:
for i in range(7):
  target = new_target[i]
  parameter = psvc[i]
  model = DecisionTreeClassifier(**parameter)

  result = model_result('Decision Three', model, X_train, y_train[target], X_test, y_test[target])
  print(result)

Baseline model(= Dummy classfier)

In [ ]:
dummy = DummyClassifier(strategy = 'most_frequent')

In [ ]:
for target in new_target:

  result = dummy_result('Dummy', dummy, X_train, y_train[target], X_test, y_test[target])
  print(result)